# GW Approximation Tutorial Notebook

In this notebook, we present an example calculation of quasiparticle energies using QuatumMASALA's `gw` module.

In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
# Imports
from __future__ import annotations
import numpy as np
import sys


sys.path.append(".")

dirname = "../../../tests/bgw/silicon/cohsex/"

### Load Input Files
Input data is handled by the ``EpsInp`` class.\
The data can be provided either by constructing the ``EpsInp`` object or by reading BGW-compatible input file ``epsilon.inp``.\
The attributes have been supplied with docstrings from BerkeleyGW's input specification, so they will be accessible directly in most IDEs.

In [17]:
from qtm.gw.io_bgw.epsinp import Epsinp

# Constructing input manually
epsinp = Epsinp(epsilon_cutoff=10.0,
                use_wfn_hdf5=True,
                number_bands=8,
                write_vcoul=True,
                qpts=[[0.0,0.0,0.0]],
                is_q0=[True])

# Reading from epsilon.inp file
epsinp = Epsinp.from_epsilon_inp(filename=dirname+'epsilon.inp')
# print(epsinp)

# There is an analogous system to read SigmaInp
from qtm.gw.io_bgw.sigmainp import Sigmainp
sigmainp = Sigmainp.from_sigma_inp(filename=dirname+'sigma.inp')
# print(sigmainp)

### Load WfnData
Calculation of dielectric matrix requires mean field eigenfunctions. \
Wavefunction data generated from mean-field codes can be read using the ``wfn2py`` utility, which assumes that the incoming data satisfies BerkeleyGW's [`wfn_h5`](http://manual.berkeleygw.org/3.0/wfn_h5_spec/) specification. The data is stored as a `NamedTuple` object.

For reasons discussed later, we also require wavefunctions on a shifted grid to calculate dielectric matrix at $q\to 0$. This shifted grid dataset will be referred to as `wfnqdata`.

Similarly, the utilities `read_rho` and `read_vxc` can be used to read density and exchange-correlation respectively.

In [18]:
# wfn2py
from qtm.gw.io_bgw import inp
from qtm.gw.io_bgw.wfn2py import wfn2py

wfndata = wfn2py(dirname+'WFN.h5')#, wfn_ecutrho_minus_ecutwfn=epsinp.epsilon_cutoff)
print(wfndata.__doc__)

wfnqdata = wfn2py(dirname+'WFNq.h5')#, wfn_ecutrho_minus_ecutwfn=epsinp.epsilon_cutoff)
# print(wfnqdata.__doc__)

# RHO data
rho = inp.read_rho(dirname+"RHO")

# Vxc data
vxc = inp.read_vxc(dirname+"vxc.dat")

WfnData(crystal, grho, kpts, l_gk, l_wfn, symmetry)


### Initialize Epsilon Class

``Epsilon`` class can be initialized by either directly passing the required `quantummasala.core` objects or by passing the input objects discussed earlier.

In [19]:
from qtm.gw.core import QPoints
from qtm.gw.epsilon import Epsilon

epsilon = Epsilon(
    crystal = wfndata.crystal,
    gspace = wfndata.grho,
    kpts = wfndata.kpts,
    kptsq = wfnqdata.kpts,
    l_wfn = wfndata.l_wfn,
    l_wfnq = wfnqdata.l_wfn,
    l_gsp_wfn = wfndata.l_gk,
    l_gsp_wfnq = wfnqdata.l_gk,
    qpts = QPoints.from_cryst(wfndata.kpts.recilat, epsinp.is_q0, *epsinp.qpts),
    epsinp = epsinp,
)

epsilon = Epsilon.from_data(wfndata=wfndata, wfnqdata=wfnqdata, epsinp=epsinp)

Vcoul calculation for qpts: 100%|██████████| 64/64 [00:00<00:00, 12373.72it/s]


The three main steps involved in the calculation have been mapped to the corresponding functions:
1.  ``matrix_elements``: Calculation of Planewave Matrix elements
    $$M_{nn'}({\textbf k},{\textbf q},{\textbf G}) = \bra{n\,{\textbf k}{+}{\textbf q}}e^{i({\textbf q}+{\textbf G})\cdot{\textbf r}}\ket{n'\,\textbf k}$$
    where the $\textbf G$-vectors included in the calculation satisfy $|\textbf q + \textbf G|^2 < E_{\text{cut}}$.
    Since this is a convolution in k-space, the time complexity can be reduced from $\mathcal{O}\left(N^2_G\right)$ to $\mathcal{O}\left(N_G\ln N_G\right)$ by using Fast Fourier Transform, where $N_G$  the number of reciprocal lattice vectors in the wavefunction.
    $$
    M_{nn'}({\bf k},{\bf q},\{{\bf G}\}) = {\rm FFT}^{-1}\left( \phi^{*}_{n,{\bf k}+{\bf q} }({\bf r}) \phi_{n',{\bf k} }({\bf r}) \right).
    $$
    where $\phi_{n',{\bf k}}({\bf r}) = {\rm FFT}\left( \psi_{n\bf k}(\bf G)\right)$. 
    
2.  ``polarizability``: Calculation of RPA polarizability matrix $P$
    $$
        P_{\textbf{GG'}}{\left({\textbf q}\;\!;0\right)}=
        \,\,{}\sum_{n}^{\textrm occ}\sum_{n'}^{\textrm emp}\sum_{{\textbf k}}
        \frac{
        \bra{n'\textbf k}e^{-i({\textbf q}+{\textbf G})\cdot{\textbf r}}\ket{n{\textbf k}{+}{\textbf q}}
        \bra{n{\textbf k}{+}{\textbf q}}e^{i({\textbf q}+{\textbf G'})\cdot{\textbf r}}\ket{n'\textbf k}
        }{E_{n{\textbf k}{+}{\textbf q}}\,{-}\,E_{n'{\textbf k}}}.
    $$
3.  ``epsilon_inverse``: Calculation of (static) epsilon-inverse matrix
    $$
        \epsilon_{\textbf{GG'}}{\left({\textbf q}\;\!\right)}=
        \delta_{\textbf{GG'}}\,{-}\,v{\left({\textbf q}{+}{\textbf G}\right)} \,
        P_{\textbf{GG'}}{\left({\textbf q}\;\!\right)}
    $$
    where $ v(\textbf{q} + \textbf{G}) = \frac{8\pi}{\left|\textbf q + \textbf G\right|^2} $ is bare Coulomb potential, written in Rydberg units. If this formula is used as-is for the case where $|\textbf q| = |\textbf G| = 0$, the resulting $v\left({\textbf{q=0}, \textbf{G=0}}\;\!\right)$ blows up as $1/q^2$. However, for 3D gapped systems, the matrix elements $\big| M_{nn'}\left({\bf k},{\textbf{q}\to\textbf{0}},{\textbf{G=0}}\right)\big| \sim q$ cancel the Coulomb divergence and $\epsilon_{\textbf{00}}\left({\textbf q\to\textbf{0}}\;\!\right) \sim q^2/q^2$ which is a finite number. In order to calculate $\epsilon_{\textbf{00}}\left({\textbf q=\textbf{0}}\;\!\right)$, we use the scheme specified in \cite{BGW2012}, wherein $q=0$ is replaced with a small but non-zero value. Since matrix element calculation involves the eigenvectors $\ket{n{\textbf k}{+}{\textbf q}}$, having a non-$\Gamma$-centered $\textbf q\to 0$ point requires mean-field eigenvectors over a shifted $k$-grid.

In [20]:
from tqdm import trange
from qtm.gw.core import reorder_2d_matrix_sorted_gvecs, sort_cryst_like_BGW


def calculate_epsilon(numq=None, writing=False):
    epsmats = []
    if numq is None:
        numq = epsilon.qpts.numq

    for i_q in trange(0, numq, desc="Epsilon> q-pt index"):
        # Create map between BGW's sorting order and QTm's sorting order
        gkspc = epsilon.l_gq[i_q]
        if i_q == epsilon.qpts.index_q0:
            key = gkspc.g_norm2
            indices_gspace_sorted = sort_cryst_like_BGW(
                cryst=gkspc.g_cryst, key_array=key
            )
        else:
            key = gkspc.gk_norm2
            indices_gspace_sorted = sort_cryst_like_BGW(
                cryst=gkspc.g_cryst, key_array=key
            )

        # Calculate polarizability matrix (Memory-inefficient, but faster)
        chimat = epsilon.polarizability(next(epsilon.matrix_elements(i_q=i_q)))

        # Calculate polarizability matrix (memory-efficient)
        # chimat = epsilon.polarizability_active(i_q)

        # Calculate epsilon inverse matrix
        epsinv = epsilon.epsilon_inverse(i_q=i_q, polarizability_matrix=chimat, store=True)

        # indices = epsilon.l_gq[i_q].gk_indices_tosorted
        epsinv = reorder_2d_matrix_sorted_gvecs(epsinv, indices_gspace_sorted)
        epsilon.l_epsinv[i_q] = epsinv
        
        # Compare the results with BGW's results
        if i_q == epsilon.qpts.index_q0:
            epsref = epsilon.read_epsmat(dirname + "eps0mat.h5")[0][0, 0]
            # indices = epsilon.l_gq[i_q].gk_indices_tosorted
            if writing:
                epsilon.write_epsmat(
                    filename="test/epsilon/eps0mat_qtm.h5", epsinvmats=[epsinv]
                )
        else:
            epsref = np.array(epsilon.read_epsmat(dirname + "epsmat.h5")[i_q - 1][0, 0])
            epsmats.append(epsinv)

        # Calculate stddev between reference and calculated epsinv matrices
        # print("epsinv.shape", epsinv.shape,"epsref.shape", epsref.shape)
        mindim = min(*epsref.shape)#, *epsinv.shape)
        epsref = epsref[:mindim, :mindim]
        std_eps = np.std(epsref - epsinv) / np.sqrt(np.prod(list(epsinv.shape)))

        epstol = 1e-15
        if np.abs(std_eps) > epstol:
            print(f"Standard deviation exceeded {epstol} tolerance", std_eps)
            print("i_q", i_q)
            # print(epsinv[0])
            # print(epsref[0])

    if writing:
        epsilon.write_epsmat(filename="test/epsilon/epsmat_qtm.h5", epsinvmats=epsmats)


epsinp.no_min_fftgrid = True
epsilon = Epsilon.from_data(wfndata=wfndata, wfnqdata=wfnqdata, epsinp=epsinp)
calculate_epsilon()#numq=1)

# epsinp.no_min_fftgrid = False
# epsilon = Epsilon.from_data(wfndata=wfndata, wfnqdata=wfnqdata, epsinp=epsinp)
# calculate_epsilon()#numq=1)


Epsilon> q-pt index: 100%|██████████| 64/64 [00:27<00:00,  2.32it/s]


### Sigma Calculation

In [21]:
from qtm.gw.sigma import Sigma


# dirname = "./scripts/results/si_6_nband272_pristine_cohsex/si_6_gw/"
# outdir = "./test/tempdir_20230618_173806/"
outdir = dirname+"test/"

sigma = Sigma.from_data(
    wfndata=wfndata,
    wfnqdata=wfnqdata,
    sigmainp=sigmainp,
    epsinp=epsinp,
    l_epsmats=epsilon.l_epsinv,
    rho=rho,
    vxc=vxc,
    outdir=dirname+"temp/",
)

Vcoul calculation for qpts: 100%|██████████| 64/64 [00:00<00:00, 15729.25it/s]


vcoul: Vcoul:
        * gspace = <qtm.gspace.gspc.GSpace object at 0x7f5e36313710>
        * qpts = <qtm.gw.core.QPoints object at 0x7f5e34b59690>
        * bare_coulomb_cutoff = 10.0
        * avgcut = 1e-05
        * l_gspace_q = <class 'list'> of length 64
        * vcoul = <class 'list'> of length 64
        * N_SAMPLES = 2500000.0
        * N_SAMPLES_COARSE = 250000.0
        * SEED = 5000
        


Vcoul calculation for qpts: 100%|██████████| 64/64 [00:55<00:00,  1.16it/s]


In [22]:
print(sigma.rho.gvecs.shape)

(4477, 3)


In [23]:
sigma.print_condition=False
sigma_x_mat = sigma.sigma_x()    
print("Sigma X GPP")
sigma.pprint_sigma_mat(np.real(sigma_x_mat.T))


Sigma_X:   0%|          | 0/64 [00:00<?, ?it/s]

Sigma_X: 100%|██████████| 64/64 [00:02<00:00, 24.69it/s]

Sigma X GPP
  -17.544906  -16.031120  -16.916213
  -12.984799  -16.031115  -14.895720
  -12.984990  -13.425014  -13.214186
  -12.984879  -13.425073  -13.214228
   -5.772040   -5.203257   -5.937991
   -5.772092   -5.202819   -5.086562
   -5.772459   -3.793211   -5.086561
   -5.801537   -3.793222   -2.418507


In [24]:
sigma.print_condition=False
sigma_sx_cohsex_mat = sigma.sigma_sx_static()    
print("Sigma SX COHSEX")
sigma.pprint_sigma_mat(np.real(sigma_sx_cohsex_mat.T))

Sigma_SX_Static:   0%|          | 0/64 [00:00<?, ?it/s]

Sigma_SX_Static: 100%|██████████| 64/64 [00:02<00:00, 25.70it/s]

Sigma SX COHSEX
   11.897822   10.692213   11.350437
    8.050415   10.691906   10.025763
    8.052588    8.736187    8.384990
    8.052483    8.736867    8.385902
    2.917532    2.597576    2.882170
    2.915433    2.597634    2.433213
    2.919747    1.333271    2.431261
    2.674705    1.334266    0.749947


In [25]:
sigma.print_condition=False
sigma_ch_cohsex_mat = sigma.sigma_ch_static()    
print("Sigma CH COHSEX")
sigma.pprint_sigma_mat(np.real(sigma_ch_cohsex_mat.T))

Sigma_CH_Static_Partial:   0%|          | 0/64 [00:00<?, ?it/s]

Sigma_CH_Static_Partial: 100%|██████████| 64/64 [00:04<00:00, 14.09it/s]

Sigma CH COHSEX
   -6.641277   -6.317907   -6.511362
   -5.113976   -6.317977   -6.076655
   -5.114200   -5.580271   -5.418842
   -5.114077   -5.580427   -5.419831
   -4.749343   -4.710424   -4.933476
   -4.748586   -4.710832   -4.288478
   -4.750098   -3.472540   -4.288100
   -4.781285   -3.473914   -2.996284


In [26]:
sigma.print_condition=False
sigma_ch_exact_mat = sigma.sigma_ch_static_exact()    
print("Sigma CH COHSEX EXACT")
sigma.pprint_sigma_mat(np.real(sigma_ch_exact_mat.T))

Sigma_CH_Static_Exact:   0%|          | 0/64 [00:00<?, ?it/s]

Sigma_CH_Static_Exact: 100%|██████████| 64/64 [00:26<00:00,  2.39it/s]

Sigma CH COHSEX EXACT
   -7.461535   -7.645228   -7.650968
   -7.788499   -7.645261   -7.324929
   -7.788918   -7.500747   -7.703410
   -7.788862   -7.501264   -7.704059
   -7.361444   -6.804707   -7.377943
   -7.360928   -6.804653   -7.142290
   -7.362404   -7.712695   -7.142081
   -7.851026   -7.714351   -6.239520


In [29]:
sigma.calculate_static_cohsex()

Sigma_X:   0%|          | 0/64 [00:00<?, ?it/s]

Sigma_X: 100%|██████████| 64/64 [00:03<00:00, 17.20it/s]


Sigma X
[[-17.544906+0.j -12.984799+0.j -12.98499 +0.j -12.984879+0.j
   -5.77204 +0.j  -5.772092+0.j  -5.772459+0.j  -5.801537+0.j]
 [-16.03112 +0.j -16.031115+0.j -13.425014+0.j -13.425073+0.j
   -5.203257+0.j  -5.202819+0.j  -3.793211+0.j  -3.793222+0.j]
 [-16.916213+0.j -14.89572 +0.j -13.214186+0.j -13.214228+0.j
   -5.937991+0.j  -5.086562+0.j  -5.086561+0.j  -2.418507+0.j]]


Sigma_SX_Static: 100%|██████████| 64/64 [00:03<00:00, 17.92it/s]


Sigma SX STATIC
[[11.897822-0.j        8.050415-0.j        8.052588+0.j
   8.052483+0.j        2.917532+0.j        2.915433+0.j
   2.919747-0.j        2.674705-0.j      ]
 [10.692213-0.002825j 10.691906+0.002825j  8.736187+0.j
   8.736867-0.j        2.597576-0.00052j   2.597634+0.00052j
   1.333271+0.j        1.334266-0.j      ]
 [11.350437-0.j       10.025763+0.j        8.38499 -0.j
   8.385902+0.j        2.88217 -0.j        2.433213+0.j
   2.431261+0.j        0.749947+0.j      ]]


Sigma_CH_Static_Partial: 100%|██████████| 64/64 [00:04<00:00, 13.48it/s]


Sigma CH STATIC
[[-6.641277+0.j       -5.113976+0.j       -5.1142  -0.j
  -5.114077-0.j       -4.749343+0.j       -4.748586-0.j
  -4.750098+0.j       -4.781285+0.j      ]
 [-6.317907+0.001465j -6.317977-0.001465j -5.580271-0.j
  -5.580427+0.j       -4.710424+0.000515j -4.710832-0.000515j
  -3.47254 -0.j       -3.473914+0.j      ]
 [-6.511362+0.j       -6.076655-0.j       -5.418842+0.j
  -5.419831-0.j       -4.933476+0.j       -4.288478+0.j
  -4.2881  -0.j       -2.996284-0.j      ]]


Sigma_CH_Static_Exact: 100%|██████████| 64/64 [00:27<00:00,  2.33it/s]

Sigma CH EXACT STATIC
[[-7.461535+0.j       -7.788499+0.j       -7.788918-0.j
  -7.788862-0.j       -7.361444+0.j       -7.360928-0.j
  -7.362404+0.j       -7.851026-0.j      ]
 [-7.645228-0.001649j -7.645261+0.001649j -7.500747+0.j
  -7.501264-0.j       -6.804707-0.000647j -6.804653+0.000647j
  -7.712695+0.j       -7.714351+0.j      ]
 [-7.650968+0.j       -7.324929-0.j       -7.70341 +0.j
  -7.704059-0.j       -7.377943+0.j       -7.14229 +0.j
  -7.142081-0.j       -6.23952 -0.j      ]]
Sig (Exact):
[[-13.108618-0.j       -12.984135-0.004475j -13.216744-0.j      ]
 [-12.722883-0.j       -12.98447 +0.004475j -12.194886+0.j      ]
 [-12.721321-0.j       -12.189574+0.j       -12.532606-0.j      ]
 [-12.721258-0.j       -12.18947 -0.j       -12.532385+0.j      ]
 [-10.215951+0.j        -9.410387-0.001167j -10.433764+0.j      ]
 [-10.217588+0.j        -9.409837+0.001167j  -9.795639+0.j      ]
 [-10.215116+0.j       -10.172634+0.j        -9.797381+0.j      ]
 [-10.977857-0.j       -10.1733

In [28]:
sigma.calculate_gpp()

Sigma_X:   0%|          | 0/64 [00:00<?, ?it/s]

Sigma_X: 100%|██████████| 64/64 [00:02<00:00, 26.27it/s]


Sigma X GPP
[[-17.544906+0.j -12.984799+0.j -12.98499 +0.j -12.984879+0.j
   -5.77204 +0.j  -5.772092+0.j  -5.772459+0.j  -5.801537+0.j]
 [-16.03112 +0.j -16.031115+0.j -13.425014+0.j -13.425073+0.j
   -5.203257+0.j  -5.202819+0.j  -3.793211+0.j  -3.793222+0.j]
 [-16.916213+0.j -14.89572 +0.j -13.214186+0.j -13.214228+0.j
   -5.937991+0.j  -5.086562+0.j  -5.086561+0.j  -2.418507+0.j]]


Sigma_CH_Static_Partial: 100%|██████████| 64/64 [00:04<00:00, 13.91it/s]


Sigma CH STATIC COHSEX
[[-6.641277+0.j       -5.113976+0.j       -5.1142  -0.j
  -5.114077-0.j       -4.749343+0.j       -4.748586-0.j
  -4.750098+0.j       -4.781285+0.j      ]
 [-6.317907+0.001465j -6.317977-0.001465j -5.580271-0.j
  -5.580427+0.j       -4.710424+0.000515j -4.710832-0.000515j
  -3.47254 -0.j       -3.473914+0.j      ]
 [-6.511362+0.j       -6.076655-0.j       -5.418842+0.j
  -5.419831-0.j       -4.933476+0.j       -4.288478+0.j
  -4.2881  -0.j       -2.996284-0.j      ]]


...

Sigma SX GPP
[[12.264463-0.000e+00j  8.203939-0.000e+00j  8.207702+0.000e+00j
   8.196115-0.000e+00j  3.356442+0.000e+00j  3.353732+0.000e+00j
   3.35863 -0.000e+00j  3.373766-0.000e+00j]
 [10.86282 -2.257e-03j 10.86242 +2.257e-03j  8.852984+1.000e-06j
   8.854197-1.000e-06j  2.869535-1.025e-03j  2.869395+1.025e-03j
   1.77997 -0.000e+00j  1.787217+0.000e+00j]
 [11.567866-0.000e+00j 10.184037+0.000e+00j  8.518146-0.000e+00j
   8.520741-0.000e+00j  3.293694-0.000e+00j  2.792918+0.000e+00j
   2.786417+0.000e+00j  1.095793-0.000e+00j]]


Sigma_CH_GPP: 100%|██████████| 64/64 [01:10<00:00,  1.10s/it]


Sigma CH GPP
<class 'numpy.ndarray'>
[[-5.809203+0.000e+00j -5.426201+0.000e+00j -5.427718-0.000e+00j
  -5.416109-0.000e+00j -5.398088-0.000e+00j -5.397232-0.000e+00j
  -5.400241+0.000e+00j -5.754373+0.000e+00j]
 [-5.862668+1.486e-03j -5.862915-1.486e-03j -5.567567-1.000e-06j
  -5.568202+1.000e-06j -4.984852+9.330e-04j -4.984614-9.330e-04j
  -4.661941+0.000e+00j -4.67023 -0.000e+00j]
 [-5.896258+0.000e+00j -5.647912-0.000e+00j -5.584283+0.000e+00j
  -5.58636 -0.000e+00j -5.448451+0.000e+00j -4.889763-0.000e+00j
  -4.885163-0.000e+00j -3.779749+0.000e+00j]]
Sig GPP:
[[-11.089642-0.00e+00j -11.031114-7.69e-04j -11.244657-0.00e+00j]
 [-10.207861-0.00e+00j -11.031756+7.69e-04j -10.359839+0.00e+00j]
 [-10.205808-0.00e+00j -10.140086+0.00e+00j -10.280927-0.00e+00j]
 [-10.205675-0.00e+00j -10.139568-0.00e+00j -10.280453-0.00e+00j]
 [ -7.81452 +0.00e+00j  -7.319359-9.10e-05j  -8.093495+0.00e+00j]
 [ -7.816428+0.00e+00j  -7.318821+9.10e-05j  -7.184402+0.00e+00j]
 [ -7.814906+0.00e+00j  -6.67656

Sigma_SX_GPP: 100%|██████████| 64/64 [01:07<00:00,  1.05s/it]

Sigma CH GPP dE
[[-6.069133+0.000e+00j -5.674153+0.000e+00j -5.675515-0.000e+00j
  -5.67074 -0.000e+00j -5.638703-0.000e+00j -5.637616-0.000e+00j
  -5.641277+0.000e+00j -5.906215+0.000e+00j]
 [-6.140514+1.592e-03j -6.140759-1.592e-03j -5.852472-1.000e-06j
  -5.853379+1.000e-06j -5.283794+1.031e-03j -5.283891-1.031e-03j
  -4.911355+0.000e+00j -4.918082-0.000e+00j]
 [-6.167574+0.000e+00j -5.920682-0.000e+00j -5.876948+0.000e+00j
  -5.879183-0.000e+00j -5.771614+0.000e+00j -5.105332-0.000e+00j
  -5.105442-0.000e+00j -4.068753-0.000e+00j]]
Sigma SX GPP dE
[[12.125756-0.000e+00j  8.26073 -0.000e+00j  8.264336+0.000e+00j
   8.259618-0.000e+00j  3.393069+0.000e+00j  3.390101+0.000e+00j
   3.39565 -0.000e+00j  3.326756-0.000e+00j]
 [10.845385-2.309e-03j 10.845035+2.309e-03j  8.913773+1.000e-06j
   8.915292-1.000e-06j  2.971232-1.098e-03j  2.971444+1.098e-03j
   1.797535-0.000e+00j  1.802831+0.000e+00j]
 [11.517813-0.000e+00j 10.175576+0.000e+00j  8.604115-0.000e+00j
   8.606857+0.000e+00j  3.4